In [ ]:
from starfysh import (AA, utils, plot_utils, post_analysis)
from starfysh import starfysh as sf_model
import scanpy as sc
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import rcParams

import seaborn as sns
sns.set_style('white')

In [ ]:
data_path = '/kaggle/input/bcdata2'
sample_id = 'BreastCancer'

In [ ]:
import gc
path = '/kaggle/input/bcdata2/BreastCancer'
def cheap_load(path):
    adata = sc.read_visium(path)
    adata.var_names_make_unique()
    raw = adata.copy()
    del adata.uns
    gc.collect()
    adata.var['mt'] = np.logical_or(adata.var_names.str.startswith('MT-'), adata.var_names.str.startswith('mt-'))
    adata.var['rb'] = (adata.var_names.str.startswith('RP-') | adata.var_names.str.startswith('rp-'))
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)
    mask_cell = adata.obs['pct_counts_mt'] < 100 #mt_thld
    mask_gene = np.logical_and(~adata.var['mt'], ~adata.var['rb'])
    adata = adata[mask_cell, mask_gene]
    sc.pp.normalize_total(adata, inplace=True)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor='seurat', n_top_genes=1000, inplace=True)
    raw = raw[adata.obs_names, adata.var_names]
    raw.var['highly_variable'] = adata.var['highly_variable']
    raw.obs = adata.obs
    return raw, adata

In [ ]:
adata, adata_normed = cheap_load(path)

In [ ]:
aa_model = AA.ArchetypalAnalysis(adata_orig=adata_normed)
archetype, arche_dict, major_idx, evs = aa_model.compute_archetypes(display=False)
aa_model.find_archetypal_spots()
gene_sig = aa_model.find_markers()
gene_sig = utils.filter_gene_sig(gene_sig, adata.to_df())
gene_sig.head()